In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-serverless

# Optimizaciones de transpilación con SABRE
*Estimación de uso: menos de un minuto en un procesador Heron r2 (NOTA: Esto es solo una estimación. Su tiempo de ejecución puede variar.)*
## Antecedentes
La transpilación es un paso crítico en Qiskit que convierte circuitos cuánticos en formas compatibles con hardware cuántico específico. Involucra dos etapas clave: **disposición de qubits** (mapeo de qubits lógicos a qubits físicos en el dispositivo) y **enrutamiento de compuertas** (asegurar que las compuertas multi-qubit respeten la conectividad del dispositivo insertando compuertas SWAP según sea necesario).

SABRE (*SWAP-Based Bidirectional heuristic search algorithm*) es una poderosa herramienta de optimización tanto para la disposición como para el enrutamiento. Es especialmente eficaz para **circuitos a gran escala** (100+ qubits) y dispositivos con mapas de acoplamiento complejos, como el **IBM&reg; Heron**, donde el crecimiento exponencial en las posibles asignaciones de qubits exige soluciones eficientes.

### ¿Por qué utilizar SABRE?
SABRE minimiza el número de compuertas SWAP y reduce la profundidad del circuito, mejorando el rendimiento del circuito en hardware real. Su enfoque basado en heurísticas lo hace ideal para hardware avanzado y circuitos grandes y complejos. Las mejoras recientes introducidas en el algoritmo [LightSABRE](https://arxiv.org/abs/2409.08368) optimizan aún más el rendimiento de SABRE, ofreciendo tiempos de ejecución más rápidos y menos compuertas SWAP. Estas mejoras lo hacen aún más eficaz para circuitos a gran escala.

### Lo que aprenderá
Este tutorial se divide en dos partes:
1. Aprenda a utilizar SABRE con **patrones de Qiskit** para la optimización avanzada de circuitos grandes.
2. Aproveche **qiskit_serverless** para maximizar el potencial de SABRE en una transpilación escalable y eficiente.

Usted podrá:
- Optimizar SABRE para circuitos con 100+ qubits, superando las configuraciones de transpilación predeterminadas como `optimization_level=3`.
- Explorar las **mejoras de LightSABRE** que mejoran el tiempo de ejecución y reducen el conteo de compuertas.
- Personalizar parámetros clave de SABRE (`swap_trials`, `layout_trials`, `max_iterations`, `heuristic`) para equilibrar la **calidad del circuito** y el **tiempo de transpilación**.
## Requisitos
Antes de comenzar este tutorial, asegúrese de tener instalado lo siguiente:
- Qiskit SDK v1.0 o posterior, con soporte de [visualización](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.28 o posterior (`pip install qiskit-ibm-runtime`)
- Serverless (`pip install qiskit-ibm-catalog qiskit_serverless`)
## Configuración

In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_catalog import QiskitServerless, QiskitFunction
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorOptions
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import matplotlib.pyplot as plt
import numpy as np
import time

## Parte I. Uso de SABRE con patrones de Qiskit

SABRE puede utilizarse en Qiskit para optimizar circuitos cuánticos al manejar tanto las etapas de disposición de qubits como de enrutamiento de compuertas. En esta sección, le guiaremos a través del **ejemplo mínimo** del uso de SABRE con patrones de Qiskit, con el enfoque principal en el paso 2 de optimización.

Para ejecutar SABRE, usted necesita:
- Una representación **DAG** (Grafo Acíclico Dirigido) de su circuito cuántico.
- El **mapa de acoplamiento** del backend, que especifica cómo los qubits están conectados físicamente.
- El **pase SABRE**, que aplica el algoritmo para optimizar la disposición y el enrutamiento.

Para esta parte, nos enfocaremos en el pase **SabreLayout**. Este realiza tanto pruebas de disposición como de enrutamiento, trabajando para encontrar la disposición inicial más eficiente mientras minimiza el número de compuertas SWAP necesarias. Es importante señalar que `SabreLayout`, por sí solo, optimiza internamente tanto la disposición como el enrutamiento al almacenar la solución que agrega la menor cantidad de compuertas SWAP. Tenga en cuenta que al usar solo **SabreLayout**, no podemos cambiar la heurística de SABRE, pero sí podemos personalizar el número de `layout_trials`.

### Paso 1: Mapear entradas clásicas a un problema cuántico

Un circuito **GHZ (Greenberger-Horne-Zeilinger)** es un circuito cuántico que prepara un estado entrelazado donde todos los qubits están en el estado `|0...0⟩` o `|1...1⟩`. El estado GHZ para $n$ qubits se representa matemáticamente como:
$$ |\text{GHZ}\rangle = \frac{1}{\sqrt{2}} \left( |0\rangle^{\otimes n} + |1\rangle^{\otimes n} \right) $$

Se construye aplicando:
1. Una compuerta Hadamard al primer qubit para crear superposición.
2. Una serie de compuertas CNOT para entrelazar los qubits restantes con el primero.

Para este ejemplo, construimos intencionalmente un **circuito GHZ de topología en estrella** en lugar de uno de topología lineal. En la topología en estrella, el primer qubit actúa como el "centro", y todos los demás qubits se entrelazan directamente con él usando compuertas CNOT. Esta elección es deliberada porque, mientras que el **estado GHZ de topología lineal** puede implementarse teóricamente en profundidad $ O(N) $ en un mapa de acoplamiento lineal sin ninguna compuerta SWAP, SABRE encontraría trivialmente una solución óptima al mapear un circuito GHZ de 100 qubits a un subgrafo del mapa de acoplamiento heavy-hex del backend.

El **circuito GHZ de topología en estrella** presenta un problema significativamente más desafiante. Aunque teóricamente todavía puede ejecutarse en profundidad $ O(N) $ sin compuertas SWAP, encontrar esta solución requiere identificar una disposición inicial óptima, lo cual es mucho más difícil debido a la conectividad no lineal del circuito. Esta topología sirve como un mejor caso de prueba para evaluar SABRE, ya que demuestra cómo los parámetros de configuración impactan el rendimiento de la disposición y el enrutamiento bajo condiciones más complejas.

![ghz_star_topology.png](../docs/images/tutorials/transpilation-optimizations-with-sabre/ghz_star_topology.avif)

Aspectos notables:
- La herramienta **HighLevelSynthesis** puede producir la solución óptima de profundidad $ O(N) $ para el circuito GHZ de topología en estrella sin introducir compuertas SWAP, como se muestra en la imagen anterior.
- Alternativamente, el pase **StarPrerouting** puede reducir aún más la profundidad al guiar las decisiones de enrutamiento de SABRE, aunque puede introducir algunas compuertas SWAP. Sin embargo, StarPrerouting aumenta el tiempo de ejecución y requiere integración en el proceso de transpilación inicial.

Para los propósitos de este tutorial, excluimos tanto HighLevelSynthesis como StarPrerouting para aislar y resaltar el impacto directo de la configuración de SABRE en el tiempo de ejecución y la profundidad del circuito. Al medir el valor esperado $ \langle Z_0 Z_i \rangle $ para cada par de qubits, analizamos:
- Qué tan bien SABRE reduce las compuertas SWAP y la profundidad del circuito.
- El efecto de estas optimizaciones en la fidelidad del circuito ejecutado, donde las desviaciones de $ \langle Z_0 Z_i \rangle = 1 $ indican pérdida de entrelazamiento.

In [2]:
# set seed for reproducibility
seed = 42
num_qubits = 110

# Create GHZ circuit
qc = QuantumCircuit(num_qubits)
qc.h(0)
for i in range(1, num_qubits):
    qc.cx(0, i)

qc.measure_all()

A continuación, mapearemos los operadores de interés para evaluar el comportamiento del sistema. Específicamente, utilizaremos operadores `ZZ` entre qubits para examinar cómo se degrada el entrelazamiento a medida que los qubits se alejan entre sí. Este análisis es crítico porque las imprecisiones en los valores esperados $\langle Z_0 Z_i \rangle$ para qubits distantes pueden revelar el impacto del ruido y los errores en la ejecución del circuito. Al estudiar estas desviaciones, obtenemos información sobre qué tan bien el circuito preserva el entrelazamiento bajo diferentes configuraciones de SABRE y con qué eficacia SABRE minimiza el impacto de las restricciones del hardware.

In [3]:
# ZZII...II, ZIZI...II, ... , ZIII...IZ
operator_strings = [
    "Z" + "I" * i + "Z" + "I" * (num_qubits - 2 - i)
    for i in range(num_qubits - 1)
]
print(operator_strings)
print(len(operator_strings))

operators = [SparsePauliOp(operator) for operator in operator_strings]

['ZZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

### Step 2: Optimize problem for quantum hardware execution

In this step, we focus on optimizing the circuit layout for execution on a specific quantum hardware device with 127 qubits. This is the main focus of the tutorial, as we perform **SABRE optimizations and transpilation** to achieve the best circuit performance. Using the `SabreLayout` pass, we determine an initial qubit mapping that minimizes the need for SWAP gates during routing. By passing the `coupling_map` of the target backend, `SabreLayout` adapts the layout to the device's connectivity constraints.

We will use `generate_preset_pass_manager` with `optimization_level=3` for the transpilation process and customize the `SabreLayout` pass with different configurations. The goal is to find a setup that produces a transpiled circuit with the **lowest size and/or depth**, demonstrating the impact of SABRE optimizations.

#### Why Are Circuit Size and Depth Important?

- **Lower size (gate count):** Reduces the number of operations, minimizing opportunities for errors to accumulate.
- **Lower depth:** Shortens the overall execution time, which is critical for avoiding decoherence and maintaining quantum state fidelity.

By optimizing these metrics, we improve the circuit’s reliability and execution accuracy on noisy quantum hardware.

Select the backend.

In [4]:
service = QiskitRuntimeService()
# backend = service.least_busy(
#    operational=True, simulator=False, min_num_qubits=127
# )
backend = service.backend("ibm_boston")
print(f"Using backend: {backend.name}")

Using backend: ibm_boston


### Paso 2: Optimizar el problema para la ejecución en hardware cuántico
En este paso, nos enfocamos en optimizar la disposición del circuito para la ejecución en un dispositivo de hardware cuántico específico con 127 qubits. Este es el enfoque principal del tutorial, ya que realizamos **optimizaciones de SABRE y transpilación** para lograr el mejor rendimiento del circuito. Usando el pase `SabreLayout`, determinamos un mapeo inicial de qubits que minimiza la necesidad de compuertas SWAP durante el enrutamiento. Al pasar el `coupling_map` del backend objetivo, `SabreLayout` adapta la disposición a las restricciones de conectividad del dispositivo.

Utilizaremos `generate_preset_pass_manager` con `optimization_level=3` para el proceso de transpilación y personalizaremos el pase `SabreLayout` con diferentes configuraciones. El objetivo es encontrar una configuración que produzca un circuito transpilado con el **menor tamaño y/o profundidad**, demostrando el impacto de las optimizaciones de SABRE.

#### ¿Por qué son importantes el tamaño y la profundidad del circuito?
- **Menor tamaño (conteo de compuertas):** Reduce el número de operaciones, minimizando las oportunidades de acumulación de errores.
- **Menor profundidad:** Acorta el tiempo total de ejecución, lo cual es crítico para evitar la decoherencia y mantener la fidelidad del estado cuántico.

Al optimizar estas métricas, mejoramos la confiabilidad y precisión de ejecución del circuito en hardware cuántico ruidoso.
Seleccione el backend.

In [5]:
# Get the coupling map from the backend
cmap = CouplingMap(backend().configuration().coupling_map)

# Create the SabreLayout passes for the custom configurations
sl_2 = SabreLayout(
    coupling_map=cmap,
    seed=seed,
    max_iterations=4,
    layout_trials=200,
    swap_trials=200,
)
sl_3 = SabreLayout(
    coupling_map=cmap,
    seed=seed,
    max_iterations=8,
    layout_trials=200,
    swap_trials=200,
)

# Create the pass managers, need to first create then configure the SabreLayout passes
pm_1 = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=seed
)
pm_2 = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=seed
)
pm_3 = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=seed
)

Now we can configure the `SabreLayout` pass in the custom pass managers. To do this we know that for the default `generate_preset_pass_manager` on `optimization_level=3`, the `SabreLayout` pass is at index 2, as `SabreLayout` occurs after `SetLayout` and `VF2Laout` passes. We can access this pass and modify its parameters.

In [6]:
pm_2.layout.replace(index=2, passes=sl_2)
pm_3.layout.replace(index=2, passes=sl_3)

Para evaluar el impacto de diferentes configuraciones en la optimización del circuito, crearemos tres gestores de pases, cada uno con configuraciones únicas para el pase `SabreLayout`. Estas configuraciones ayudan a analizar el equilibrio entre la calidad del circuito y el tiempo de transpilación.

#### Parámetros clave
- **`max_iterations`**: El número de iteraciones de enrutamiento hacia adelante y hacia atrás para refinar la disposición y reducir los costos de enrutamiento.
- **`layout_trials`**: El número de disposiciones iniciales aleatorias probadas, seleccionando la que minimiza las compuertas SWAP.
- **`swap_trials`**: El número de pruebas de enrutamiento para cada disposición, refinando la ubicación de compuertas para un mejor enrutamiento.

Aumente `layout_trials` y `swap_trials` para realizar una optimización más exhaustiva, a costa de un mayor tiempo de transpilación.

#### Configuraciones en este tutorial
1. **`pm_1`**: Configuración predeterminada con `optimization_level=3`.
   - `max_iterations=4`
   - `layout_trials=20`
   - `swap_trials=20`

2. **`pm_2`**: Aumenta el número de pruebas para una mejor exploración.
   - `max_iterations=4`
   - `layout_trials=200`
   - `swap_trials=200`

3. **`pm_3`**: Extiende `pm_2` aumentando el número de iteraciones para un mayor refinamiento.
   - `max_iterations=8`
   - `layout_trials=200`
   - `swap_trials=200`

Al comparar los resultados de estas configuraciones, buscamos determinar cuál logra el mejor equilibrio entre la calidad del circuito (por ejemplo, tamaño y profundidad) y el costo computacional.

In [7]:
# Transpile the circuit with each pass manager and measure the time
t0 = time.time()
tqc_1 = pm_1.run(qc)
t1 = time.time() - t0
t0 = time.time()
tqc_2 = pm_2.run(qc)
t2 = time.time() - t0
t0 = time.time()
tqc_3 = pm_3.run(qc)
t3 = time.time() - t0

# Obtain the depths and the total number of gates (circuit size)
depth_1 = tqc_1.depth(lambda x: x.operation.num_qubits == 2)
depth_2 = tqc_2.depth(lambda x: x.operation.num_qubits == 2)
depth_3 = tqc_3.depth(lambda x: x.operation.num_qubits == 2)
size_1 = tqc_1.size()
size_2 = tqc_2.size()
size_3 = tqc_3.size()

# Transform the observables to match the backend's ISA
operators_list_1 = [op.apply_layout(tqc_1.layout) for op in operators]
operators_list_2 = [op.apply_layout(tqc_2.layout) for op in operators]
operators_list_3 = [op.apply_layout(tqc_3.layout) for op in operators]

# Compute improvements compared to pass manager 1 (default)
depth_improvement_2 = ((depth_1 - depth_2) / depth_1) * 100
depth_improvement_3 = ((depth_1 - depth_3) / depth_1) * 100
size_improvement_2 = ((size_1 - size_2) / size_1) * 100
size_improvement_3 = ((size_1 - size_3) / size_1) * 100
time_increase_2 = ((t2 - t1) / t1) * 100
time_increase_3 = ((t3 - t1) / t1) * 100

print(
    f"Pass manager 1 (4,20,20)  : Depth {depth_1}, Size {size_1}, Time {t1:.4f} s"
)
print(
    f"Pass manager 2 (4,200,200): Depth {depth_2}, Size {size_2}, Time {t2:.4f} s"
)
print(f"  - Depth improvement: {depth_improvement_2:.2f}%")
print(f"  - Size improvement: {size_improvement_2:.2f}%")
print(f"  - Time increase: {time_increase_2:.2f}%")
print(
    f"Pass manager 3 (8,200,200): Depth {depth_3}, Size {size_3}, Time {t3:.4f} s"
)
print(f"  - Depth improvement: {depth_improvement_3:.2f}%")
print(f"  - Size improvement: {size_improvement_3:.2f}%")
print(f"  - Time increase: {time_increase_3:.2f}%")

Pass manager 1 (4,20,20)  : Depth 439, Size 2346, Time 0.5775 s
Pass manager 2 (4,200,200): Depth 395, Size 2070, Time 3.9927 s
  - Depth improvement: 10.02%
  - Size improvement: 11.76%
  - Time increase: 591.43%
Pass manager 3 (8,200,200): Depth 375, Size 1873, Time 2.3079 s
  - Depth improvement: 14.58%
  - Size improvement: 20.16%
  - Time increase: 299.67%


Ahora podemos configurar el pase `SabreLayout` en los gestores de pases personalizados. Para ello, sabemos que en el `generate_preset_pass_manager` predeterminado con `optimization_level=3`, el pase `SabreLayout` está en el índice 2, ya que `SabreLayout` ocurre después de los pases `SetLayout` y `VF2Laout`. Podemos acceder a este pase y modificar sus parámetros.

In [8]:
# Plot the results of the metrics
times = [t1, t2, t3]
depths = [depth_1, depth_2, depth_3]
sizes = [size_1, size_2, size_3]
pm_names = [
    "pm_1 (4 iter, 20 trials)",
    "pm_2 (4 iter, 200 trials)",
    "pm_3 (8 iter, 200 trials)",
]
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(pm_names)))

# Create a figure with three subplots
fig, axs = plt.subplots(3, 1, figsize=(6, 9), sharex=True)
axs[0].bar(pm_names, times, color=colors)
axs[0].set_ylabel("Time (s)", fontsize=12)
axs[0].set_title("Transpilation Time", fontsize=14)
axs[0].grid(axis="y", linestyle="--", alpha=0.7)
axs[1].bar(pm_names, depths, color=colors)
axs[1].set_ylabel("Depth", fontsize=12)
axs[1].set_title("Circuit Depth", fontsize=14)
axs[1].grid(axis="y", linestyle="--", alpha=0.7)
axs[2].bar(pm_names, sizes, color=colors)
axs[2].set_ylabel("Size", fontsize=12)
axs[2].set_title("Circuit Size", fontsize=14)
axs[2].set_xticks(range(len(pm_names)))
axs[2].set_xticklabels(pm_names, fontsize=10, rotation=15)
axs[2].grid(axis="y", linestyle="--", alpha=0.7)

# Add some spacing between subplots
plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/transpilation-optimizations-with-sabre/extracted-outputs/818a8997-d2c7-4661-a6ea-f58eac376bf8-0.avif" alt="Output of the previous code cell" />

Con cada gestor de pases configurado, ahora ejecutaremos el proceso de transpilación para cada uno. Para comparar los resultados, rastrearemos métricas clave, incluyendo el tiempo de transpilación, la profundidad del circuito (medida como la profundidad de compuertas de dos qubits) y el número total de compuertas en los circuitos transpilados.

In [9]:
options = EstimatorOptions()
options.resilience_level = 2
options.dynamical_decoupling.enable = True
options.dynamical_decoupling.sequence_type = "XY4"

# Create an Estimator object
estimator = Estimator(backend, options=options)

In [10]:
# Submit the circuit to Estimator
job_1 = estimator.run([(tqc_1, operators_list_1)])
job_1_id = job_1.job_id()
print(job_1_id)

job_2 = estimator.run([(tqc_2, operators_list_2)])
job_2_id = job_2.job_id()
print(job_2_id)

job_3 = estimator.run([(tqc_3, operators_list_3)])
job_3_id = job_3.job_id()
print(job_3_id)

d5k0qs7853es738dab6g
d5k0qsf853es738dab70
d5k0qsf853es738dab7g


In [11]:
# Run the jobs
result_1 = job_1.result()[0]
print("Job 1 done")
result_2 = job_2.result()[0]
print("Job 2 done")
result_3 = job_3.result()[0]
print("Job 3 done")

Job 1 done
Job 2 done
Job 3 done


![Output of the previous code cell](../docs/images/tutorials/transpilation-optimizations-with-sabre/extracted-outputs/818a8997-d2c7-4661-a6ea-f58eac376bf8-0.avif)

### Paso 3: Ejecutar usando primitivas de Qiskit
En este paso, utilizamos la primitiva `Estimator` para calcular los valores esperados $\langle Z_0 Z_i \rangle$ para los operadores `ZZ`, evaluando el entrelazamiento y la calidad de ejecución de los circuitos transpilados. Para alinearnos con los flujos de trabajo típicos del usuario, enviamos el trabajo para ejecución y aplicamos supresión de errores usando **desacoplamiento dinámico**, una técnica que mitiga la decoherencia insertando secuencias de compuertas para preservar los estados de los qubits. Además, especificamos un nivel de resiliencia para contrarrestar el ruido, donde niveles más altos proporcionan resultados más precisos a costa de un mayor tiempo de procesamiento. Este enfoque evalúa el rendimiento de cada configuración del gestor de pases bajo condiciones de ejecución realistas.

In [12]:
data = list(range(1, len(operators) + 1))  # Distance between the Z operators

values_1 = list(result_1.data.evs)
values_2 = list(result_2.data.evs)
values_3 = list(result_3.data.evs)

plt.plot(
    data,
    values_1,
    marker="o",
    label="pm_1 (iters=4, swap_trials=20, layout_trials=20)",
)
plt.plot(
    data,
    values_2,
    marker="s",
    label="pm_2 (iters=4, swap_trials=200, layout_trials=200)",
)
plt.plot(
    data,
    values_3,
    marker="^",
    label="pm_3 (iters=8, swap_trials=200, layout_trials=200)",
)
plt.xlabel("Distance between qubits $i$")
plt.ylabel(r"$\langle Z_i Z_0 \rangle / \langle Z_1 Z_0 \rangle $")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/transpilation-optimizations-with-sabre/extracted-outputs/bc6cb36f-4bf2-4275-baf5-9557fcba520a-0.avif" alt="Output of the previous code cell" />

### Analysis of Results

The plot shows the expectation values $\langle Z_0 Z_i \rangle / \langle Z_0 Z_0 \rangle$  as a function of the distance between qubits for three pass manager configurations with increasing levels of optimization. In the ideal case, these values remain close to 1, indicating strong correlations across the circuit. As the distance increases, noise and accumulated errors lead to a decay in correlations, revealing how well each transpilation strategy preserves the underlying structure of the state.

Among the three configurations, `pm_1` clearly performs the worst. Its correlation values decay rapidly as the distance increases and approach zero much earlier than the other two configurations. This behavior is consistent with its larger circuit depth and gate count, where accumulated noise quickly degrades long-range correlations.

Both `pm_2` and `pm_3` represent significant improvements over `pm_1` across essentially all distances. On average, `pm_3` exhibits the strongest overall performance, maintaining higher correlation values over longer distances and showing a more gradual decay. This aligns with its more aggressive optimization, which produces shallower circuits that are generally more robust to noise accumulation.

That said, `pm_2` shows noticeably better accuracy at short distances compared to `pm_3`, despite having a slightly larger depth and gate count. This suggests that circuit depth alone does not fully determine performance; the specific structure produced by the transpilation, including how entangling gates are arranged and how errors propagate through the circuit, also plays an important role. In some cases, the transformations applied by `pm_2` appear to better preserve local correlations, even if they do not scale as well to longer distances.

Taken together, these results highlight a trade-off between circuit compactness and circuit structure. While increased optimization generally improves long-range stability, the best performance for a given observable depends on both reducing circuit depth and producing a structure that is well matched to the noise characteristics of the hardware.

## Part II. Configuring the heuristic in SABRE and using Serverless

In addition to adjusting trial numbers, SABRE supports customization of the routing heuristic used during transpilation. By default, `SabreLayout` employs the decay heuristic, which dynamically weights qubits based on their likelihood of being swapped. To use a different heuristic (such as the `lookahead` heuristic), you can create a custom `SabreSwap` pass and connect it to `SabreLayout` by running a `PassManager` with `FullAncillaAllocation`, `EnlargeWithAncilla`, and `ApplyLayout`. When using `SabreSwap` as a parameter for `SabreLayout`, only one layout trial is performed by default. To efficiently run multiple layout trials, we leverage the serverless runtime for parallelization. For more about serverless, see the [Serverless documentation](/docs/guides/serverless).

### How to Change the Routing Heuristic
1. Create a custom `SabreSwap` pass with the desired heuristic.
2. Use this custom `SabreSwap` as the routing method for the `SabreLayout` pass.

While it is possible to run multiple layout trials using a loop, serverless runtime is the better choice for large-scale and more vigorous experiments. Serverless supports parallel execution of layout trials, significantly speeding up the optimization of larger circuits and large experimental sweeps. This makes it especially valuable when working with resource-intensive tasks or when time efficiency is critical.

This section focuses solely on step 2 of optimization: minimizing circuit size and depth to achieve the best possible transpiled circuit. Building on the earlier results, we now explore how heuristic customization and serverless parallelization can further enhance optimization performance, making it suitable for large-scale quantum circuit transpilation.

### Results without serverless runtime (1 layout trial):

In [17]:
swap_trials = 1000

# Default PassManager with `SabreLayout` and `SabreSwap`, using heuristic "decay"
sr_default = SabreSwap(
    coupling_map=cmap, heuristic="decay", trials=swap_trials, seed=seed
)
sl_default = SabreLayout(
    coupling_map=cmap, routing_pass=sr_default, seed=seed
)
pm_default = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=seed
)
pm_default.layout.replace(index=2, passes=sl_default)
pm_default.routing.replace(index=1, passes=sr_default)

t0 = time.time()
tqc_default = pm_default.run(qc)
t_default = time.time() - t0
size_default = tqc_default.size()
depth_default = tqc_default.depth(lambda x: x.operation.num_qubits == 2)


# Custom PassManager with `SabreLayout` and `SabreSwap`, using heuristic "lookahead"
sr_custom = SabreSwap(
    coupling_map=cmap, heuristic="lookahead", trials=swap_trials, seed=seed
)
sl_custom = SabreLayout(coupling_map=cmap, routing_pass=sr_custom, seed=seed)
pm_custom = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=seed
)
pm_custom.layout.replace(index=2, passes=sl_custom)
pm_custom.routing.replace(index=1, passes=sr_custom)

t0 = time.time()
tqc_custom = pm_custom.run(qc)
t_custom = time.time() - t0
size_custom = tqc_custom.size()
depth_custom = tqc_custom.depth(lambda x: x.operation.num_qubits == 2)

print(
    f"Default (heuristic='decay')    : Depth {depth_default}, Size {size_default}, Time {t_default}"
)
print(
    f"Custom  (heuristic='lookahead'): Depth {depth_custom}, Size {size_custom}, Time {t_custom}"
)

Default (heuristic='decay')    : Depth 443, Size 3115, Time 1.034372091293335
Custom  (heuristic='lookahead'): Depth 432, Size 2856, Time 0.6669301986694336


Here we see that the `lookahead` heuristic performs better than the `decay` heuristic in terms of circuit depth, size, and time. This improvements highlights how we can improve SABRE beyond just trials and iterations for your specific circuit and hardware constraints. Note that these results are based on a single layout trial. To achieve more accurate results, we recommend running multiple layout trials, which can be done efficiently using the serverless runtime.

### Results with serverless runtime (multiple layout trials)

Qiskit Serverless requires setting up your workload’s `.py` files into a dedicated directory. The following code cell is a Python file in the `source_files` directory named `transpile_remote.py`. This file contains the function that runs the transpilation process.

In [18]:
# This cell is hidden from users, it makes sure the `source_files` directory exists
from pathlib import Path

Path("source_files").mkdir(exist_ok=True)

In [26]:
%%writefile source_files/transpile_remote.py
import time
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler import CouplingMap
from qiskit_serverless import get_arguments, save_result, distribute_task, get
from qiskit_ibm_runtime import QiskitRuntimeService

@distribute_task(target={
    "cpu": 1,
    "mem": 1024 * 1024 * 1024
})
def transpile_remote(qc, optimization_level, backend_name, seed, swap_trials, heuristic):
    """Transpiles an abstract circuit into an ISA circuit for a given backend."""

    service = QiskitRuntimeService()
    backend = service.backend(backend_name)

    pm = generate_preset_pass_manager(
        optimization_level=optimization_level,
        backend=backend,
        seed_transpiler=seed
    )

    # Changing the `SabreLayout` and `SabreSwap` passes to use the custom configurations
    cmap = CouplingMap(backend().configuration().coupling_map)
    sr = SabreSwap(coupling_map=cmap, heuristic=heuristic, trials=swap_trials, seed=seed)
    sl = SabreLayout(coupling_map=cmap, routing_pass=sr, seed=seed)
    pm.layout.replace(index=2, passes=sl)
    pm.routing.replace(index=1, passes=sr)

    # Measure the transpile time
    start_time = time.time()  # Start timer
    tqc = pm.run(qc)  # Transpile the circuit
    end_time = time.time()  # End timer

    transpile_time = end_time - start_time  # Calculate the elapsed time
    return tqc, transpile_time  # Return both the transpiled circuit and the transpile time


# Get program arguments
arguments = get_arguments()
circuit = arguments.get("circuit")
backend_name = arguments.get("backend_name")
optimization_level = arguments.get("optimization_level")
seed_list = arguments.get("seed_list")
swap_trials = arguments.get("swap_trials")
heuristic = arguments.get("heuristic")

# Transpile the circuits
transpile_worker_references = [
    transpile_remote(circuit, optimization_level, backend_name, seed, swap_trials, heuristic)
    for seed in seed_list
]

results_with_times = get(transpile_worker_references)

# Separate the transpiled circuits and their transpile times
transpiled_circuits = [result[0] for result in results_with_times]
transpile_times = [result[1] for result in results_with_times]

# Save both results and transpile times
save_result({"transpiled_circuits": transpiled_circuits, "transpile_times": transpile_times})

Overwriting source_files/transpile_remote.py


The following cell uploads the `transpile_remote.py` file as a Qiskit Serverless program under the name `transpile_remote_serverless`.

In [27]:
serverless = QiskitServerless()

transpile_remote_demo = QiskitFunction(
    title="transpile_remote_serverless",
    entrypoint="transpile_remote.py",
    working_dir="./source_files/",
)
serverless.upload(transpile_remote_demo)
transpile_remote_serverless = serverless.load("transpile_remote_serverless")

### Paso 4: Post-procesar y devolver el resultado en el formato clásico deseado
Una vez que el trabajo se completa, analizamos los resultados graficando los valores esperados $\langle Z_0 Z_i \rangle$ para cada qubit. En una simulación ideal, todos los valores $\langle Z_0 Z_i \rangle$ deberían ser 1, reflejando un entrelazamiento perfecto entre todos los qubits. Sin embargo, debido al ruido y las restricciones del hardware, los valores esperados típicamente disminuyen a medida que `i` aumenta, revelando cómo se degrada el entrelazamiento con la distancia.

En este paso, comparamos los resultados de cada configuración del gestor de pases con la simulación ideal. Al examinar la desviación de $\langle Z_0 Z_i \rangle$ respecto a 1 para cada configuración, podemos cuantificar qué tan bien cada gestor de pases preserva el entrelazamiento y mitiga los efectos del ruido. Este análisis evalúa directamente el impacto de las optimizaciones de SABRE en la fidelidad de ejecución y destaca qué configuración equilibra mejor la calidad de optimización y el rendimiento de ejecución.

Los resultados se visualizarán para resaltar las diferencias entre los gestores de pases, mostrando cómo las mejoras en la disposición y el enrutamiento influyen en la ejecución final del circuito en hardware cuántico ruidoso.

In [28]:
num_seeds = 20  # represents the different layout trials
seed_list = [seed + i for i in range(num_seeds)]

![Output of the previous code cell](../docs/images/tutorials/transpilation-optimizations-with-sabre/extracted-outputs/bc6cb36f-4bf2-4275-baf5-9557fcba520a-0.avif)

### Análisis de resultados
El gráfico muestra los valores esperados $\langle Z_0 Z_i \rangle / \langle Z_0 Z_0 \rangle$ como función de la distancia entre qubits para tres configuraciones del gestor de pases con niveles crecientes de optimización. En el caso ideal, estos valores permanecen cercanos a 1, indicando correlaciones fuertes a lo largo del circuito. A medida que la distancia aumenta, el ruido y los errores acumulados conducen a un decaimiento en las correlaciones, revelando qué tan bien cada estrategia de transpilación preserva la estructura subyacente del estado.

Entre las tres configuraciones, `pm_1` claramente tiene el peor rendimiento. Sus valores de correlación decaen rápidamente a medida que la distancia aumenta y se aproximan a cero mucho antes que las otras dos configuraciones. Este comportamiento es consistente con su mayor profundidad de circuito y conteo de compuertas, donde el ruido acumulado degrada rápidamente las correlaciones de largo alcance.

Tanto `pm_2` como `pm_3` representan mejoras significativas sobre `pm_1` en esencialmente todas las distancias. En promedio, `pm_3` exhibe el rendimiento general más fuerte, manteniendo valores de correlación más altos a distancias mayores y mostrando un decaimiento más gradual. Esto se alinea con su optimización más agresiva, que produce circuitos menos profundos que son generalmente más robustos ante la acumulación de ruido.

Dicho esto, `pm_2` muestra una precisión notablemente mejor a distancias cortas en comparación con `pm_3`, a pesar de tener una profundidad y conteo de compuertas ligeramente mayores. Esto sugiere que la profundidad del circuito por sí sola no determina completamente el rendimiento; la estructura específica producida por la transpilación, incluyendo cómo se organizan las compuertas de entrelazamiento y cómo se propagan los errores a través del circuito, también juega un papel importante. En algunos casos, las transformaciones aplicadas por `pm_2` parecen preservar mejor las correlaciones locales, incluso si no escalan tan bien a distancias mayores.

En conjunto, estos resultados destacan un equilibrio entre la compacidad del circuito y la estructura del circuito. Aunque una mayor optimización generalmente mejora la estabilidad a largo alcance, el mejor rendimiento para un observable dado depende tanto de reducir la profundidad del circuito como de producir una estructura que se ajuste bien a las características de ruido del hardware.
## Parte II. Configuración de la heurística en SABRE y uso de Serverless
Además de ajustar el número de pruebas, SABRE admite la personalización de la heurística de enrutamiento utilizada durante la transpilación. De forma predeterminada, `SabreLayout` emplea la heurística de decaimiento, que pondera dinámicamente los qubits según su probabilidad de ser intercambiados. Para utilizar una heurística diferente (como la heurística `lookahead`), puede crear un pase `SabreSwap` personalizado y conectarlo a `SabreLayout` ejecutando un `PassManager` con `FullAncillaAllocation`, `EnlargeWithAncilla` y `ApplyLayout`. Al utilizar `SabreSwap` como parámetro para `SabreLayout`, solo se realiza una prueba de disposición de forma predeterminada. Para ejecutar eficientemente múltiples pruebas de disposición, aprovechamos el entorno de ejecución serverless para la paralelización. Para obtener más información sobre serverless, consulte la [documentación de Serverless](/guides/serverless).

### Cómo cambiar la heurística de enrutamiento
1. Cree un pase `SabreSwap` personalizado con la heurística deseada.
2. Utilice este `SabreSwap` personalizado como método de enrutamiento para el pase `SabreLayout`.

Aunque es posible ejecutar múltiples pruebas de disposición usando un bucle, el entorno de ejecución serverless es la mejor opción para experimentos a gran escala y más rigurosos. Serverless admite la ejecución paralela de pruebas de disposición, acelerando significativamente la optimización de circuitos más grandes y barridos experimentales extensos. Esto lo hace especialmente valioso cuando se trabaja con tareas intensivas en recursos o cuando la eficiencia temporal es crítica.

Esta sección se enfoca únicamente en el paso 2 de optimización: minimizar el tamaño y la profundidad del circuito para lograr el mejor circuito transpilado posible. Basándonos en los resultados anteriores, ahora exploramos cómo la personalización de heurísticas y la paralelización serverless pueden mejorar aún más el rendimiento de la optimización, haciéndolo adecuado para la transpilación de circuitos cuánticos a gran escala.
### Resultados sin entorno de ejecución serverless (1 prueba de disposición):

In [29]:
job_lookahead = transpile_remote_serverless.run(
    circuit=qc,
    backend_name=backend.name,
    optimization_level=3,
    seed_list=seed_list,
    swap_trials=swap_trials,
    heuristic="lookahead",
)

In [30]:
job_lookahead.job_id

'15767dfc-e71d-4720-94d6-9212f72334c2'

In [31]:
job_lookahead.status()

'QUEUED'

Receive the logs and results from the serverless runtime.

In [21]:
logs_lookahead = job_lookahead.logs()
print(logs_lookahead)

No logs yet.


Once a program is `DONE`, you can use `job.results()` to fetch the result stored in `save_result()`.

In [32]:
# Run the job with lookahead heuristic
start_time = time.time()
results_lookahead = job_lookahead.result()
end_time = time.time()

job_lookahead_time = end_time - start_time

La siguiente celda carga el archivo `transpile_remote.py` como un programa de Qiskit Serverless con el nombre `transpile_remote_serverless`.

In [33]:
job_decay = transpile_remote_serverless.run(
    circuit=qc,
    backend_name=backend.name,
    optimization_level=3,
    seed_list=seed_list,
    swap_trials=swap_trials,
    heuristic="decay",
)

In [34]:
job_decay.job_id

'00418c76-d6ec-4bd8-9f70-05d0fa14d4eb'

In [35]:
logs_decay = job_decay.logs()
print(logs_decay)

No logs yet.


In [36]:
# Run the job with the decay heuristic
start_time = time.time()
results_decay = job_decay.result()
end_time = time.time()

job_decay_time = end_time - start_time

In [37]:
# Extract transpilation times
transpile_times_decay = results_decay["transpile_times"]
transpile_times_lookahead = results_lookahead["transpile_times"]

# Calculate total transpilation time for serial execution
total_transpile_time_decay = sum(transpile_times_decay)
total_transpile_time_lookahead = sum(transpile_times_lookahead)

# Print total transpilation time
print("=== Total Transpilation Time (Serial Execution) ===")
print(f"Decay Heuristic    : {total_transpile_time_decay:.2f} seconds")
print(f"Lookahead Heuristic: {total_transpile_time_lookahead:.2f} seconds")

# Print serverless job time (parallel execution)
print("\n=== Serverless Job Time (Parallel Execution) ===")
print(f"Decay Heuristic    : {job_decay_time:.2f} seconds")
print(f"Lookahead Heuristic: {job_lookahead_time:.2f} seconds")

# Calculate and print average runtime per transpilation
avg_transpile_time_decay = total_transpile_time_decay / num_seeds
avg_transpile_time_lookahead = total_transpile_time_lookahead / num_seeds
avg_job_time_decay = job_decay_time / num_seeds
avg_job_time_lookahead = job_lookahead_time / num_seeds

print("\n=== Average Time Per Transpilation ===")
print(f"Decay Heuristic (Serial)    : {avg_transpile_time_decay:.2f} seconds")
print(f"Decay Heuristic (Serverless): {avg_job_time_decay:.2f} seconds")
print(
    f"Lookahead Heuristic (Serial)    : {avg_transpile_time_lookahead:.2f} seconds"
)
print(
    f"Lookahead Heuristic (Serverless): {avg_job_time_lookahead:.2f} seconds"
)

# Calculate and print serverless improvement percentage
decay_improvement_percentage = (
    (total_transpile_time_decay - job_decay_time) / total_transpile_time_decay
) * 100
lookahead_improvement_percentage = (
    (total_transpile_time_lookahead - job_lookahead_time)
    / total_transpile_time_lookahead
) * 100

print("\n=== Serverless Improvement ===")
print(f"Decay Heuristic    : {decay_improvement_percentage:.2f}%")
print(f"Lookahead Heuristic: {lookahead_improvement_percentage:.2f}%")

=== Total Transpilation Time (Serial Execution) ===
Decay Heuristic    : 112.37 seconds
Lookahead Heuristic: 85.37 seconds

=== Serverless Job Time (Parallel Execution) ===
Decay Heuristic    : 5.72 seconds
Lookahead Heuristic: 5.85 seconds

=== Average Time Per Transpilation ===
Decay Heuristic (Serial)    : 5.62 seconds
Decay Heuristic (Serverless): 0.29 seconds
Lookahead Heuristic (Serial)    : 4.27 seconds
Lookahead Heuristic (Serverless): 0.29 seconds

=== Serverless Improvement ===
Decay Heuristic    : 94.91%
Lookahead Heuristic: 93.14%


These results demonstrate the substantial efficiency gains from using serverless execution for quantum circuit transpilation. Compared to serial execution, serverless execution dramatically reduces overall runtime for both the `decay` and `lookahead` heuristics by parallelizing independent transpilation trials. While serial execution reflects the full cumulative cost of exploring multiple layout trials, the serverless job times highlight how parallel execution collapses this cost into a much shorter wall-clock time. As a result, the effective time per transpilation is reduced to a small fraction of that required in the serial setting, largely independent of the heuristic used. This capability is particularly important for optimizing SABRE to its fullest potential. Many of SABRE’s strongest performance gains come from increasing the number of layout and routing trials, which can be prohibitively expensive when executed sequentially. Serverless execution removes this bottleneck, enabling large-scale parameter sweeps and deeper exploration of heuristic configurations with minimal overhead.

Overall, these findings show that serverless execution is key to scaling SABRE optimization, making aggressive experimentation and refinement practical compared to serial execution.

Obtain the results from the serverless runtime and compare the results of the lookahead and decay heuristics. We will compare the sizes and depths.

In [38]:
# Extract sizes and depths
sizes_lookahead = [
    circuit.size() for circuit in results_lookahead["transpiled_circuits"]
]
depths_lookahead = [
    circuit.depth(lambda x: x.operation.num_qubits == 2)
    for circuit in results_lookahead["transpiled_circuits"]
]
sizes_decay = [
    circuit.size() for circuit in results_decay["transpiled_circuits"]
]
depths_decay = [
    circuit.depth(lambda x: x.operation.num_qubits == 2)
    for circuit in results_decay["transpiled_circuits"]
]


def create_scatterplot(x, y1, y2, xlabel, ylabel, title, labels, colors):
    plt.figure(figsize=(8, 5))
    plt.scatter(
        x, y1, label=labels[0], color=colors[0], alpha=0.8, edgecolor="k"
    )
    plt.scatter(
        x, y2, label=labels[1], color=colors[1], alpha=0.8, edgecolor="k"
    )
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.title(title, fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()


create_scatterplot(
    seed_list,
    sizes_lookahead,
    sizes_decay,
    "Seed",
    "Size",
    "Circuit Size",
    ["lookahead", "Decay"],
    ["blue", "red"],
)
create_scatterplot(
    seed_list,
    depths_lookahead,
    depths_decay,
    "Seed",
    "Depth",
    "Circuit Depth",
    ["lookahead", "Decay"],
    ["blue", "red"],
)

<Image src="../docs/images/tutorials/transpilation-optimizations-with-sabre/extracted-outputs/4cf9588b-8ea6-4761-b544-14bef8f0be85-0.avif" alt="Output of the previous code cell" />

<Image src="../docs/images/tutorials/transpilation-optimizations-with-sabre/extracted-outputs/4cf9588b-8ea6-4761-b544-14bef8f0be85-1.avif" alt="Output of the previous code cell" />

Each point in the scatter plots above represents a layout trial, with the x-axis indicating the circuit depth and the y-axis indicating the circuit size. The results reveal that the lookahead heuristic generally outperforms the decay heuristic in minimizing circuit depth and circuit size. In practical applications, the goal is to identify the optimal layout trial for your chosen heuristic, whether prioritizing depth or size. This can be achieved by selecting the trial with the lowest value for the desired metric. Importantly, increasing the number of layout trials improves the chances of achieving a better result in terms of size or depth, but it comes at the cost of higher computational overhead.

In [39]:
min_depth_lookahead = min(depths_lookahead)
min_depth_decay = min(depths_decay)
min_size_lookahead = min(sizes_lookahead)
min_size_decay = min(sizes_decay)
print(
    "Lookahead: Min Depth",
    min_depth_lookahead,
    "Min Size",
    min_size_lookahead,
)
print("Decay:     Min Depth", min_depth_decay, "Min Size", min_size_decay)

Lookahead: Min Depth 399 Min Size 2452
Decay:     Min Depth 415 Min Size 2611


Reciba los registros y resultados del entorno de ejecución serverless.

In [40]:
# This cell is hidden from users, it cleans up the `source_files` directory
from pathlib import Path

Path("source_files/transpile_remote.py").unlink()
Path("source_files").rmdir()

## Conclusion

In this tutorial, we explored how to optimize large circuits using SABRE in Qiskit. We demonstrated how to configure the `SabreLayout` pass with different parameters to balance circuit quality and transpilation runtime. We also showed how to customize the routing heuristic in SABRE and use the `QiskitServerless`runtime to parallelize layout trials efficiently for when `SabreSwap` is involved. By adjusting these parameters and heuristics, you can optimize the layout and routing of large circuits, ensuring they are executed efficiently on quantum hardware.

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_d9YWUSQIAvU9HXE)